In [46]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)


In [48]:
def load_images_from_folder(folder, num_descriptors=10, resize_shape=(128, 128)):
    images = []
    features_glcm = []
    features_fourier = []
    labels = []
    for label, class_folder in enumerate(os.listdir(folder)):
        class_path = os.path.join(folder, class_folder)
        for filename in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                # Resize the image to a fixed shape
                img = cv2.resize(img, resize_shape)
                images.append(img)
                # Compute GLCM features
                #glcm_features = compute_glcm_features(img)
                #features_glcm.append(glcm_features)
                labels.append(label)
    # Check the length of all glcm features and crop them if necessary
    #min_length = min(len(feature) for feature in features_glcm)
    #features_glcm = [feature[:min_length] for feature in features_glcm]
    return np.array(images), np.array(labels)
x1, y1 = load_images_from_folder("/content/drive/MyDrive/archive2/Data/train")
x2, y2 = load_images_from_folder("/content/drive/MyDrive/archive2/Data/test")

In [49]:
class AntColony:
    def __init__(self, data, n_ants, n_iterations):
        self.data = data
        self.n_ants = n_ants
        self.n_iterations = n_iterations

    def _fitness_function(self, selected_features):
        selected_data = self.data[:, selected_features.astype(bool)]
        scores = np.sum(selected_data, axis=1)
        return np.mean(scores)

    def run(self, n_features):
        best_features = None
        best_score = float('-inf')

        for _ in range(self.n_iterations):
            ants = np.random.randint(2, size=(self.n_ants, self.data.shape[1]))  # Random initial feature selection
            scores = np.array([self._fitness_function(ant) for ant in ants])

            # Select top n features based on scores
            sorted_indices = np.argsort(scores)[::-1][:n_features]
            best_ant_index = sorted_indices[0]

            if scores[best_ant_index] > best_score:
                best_score = scores[best_ant_index]
                best_features = ants[best_ant_index]

        # Extract the indices of selected features
        selected_indices = np.where(best_features == 1)[0]

        # Choose the first n_features if more features were selected
        selected_indices = selected_indices[:n_features]

        return selected_indices


In [50]:
def get_block(images, num_blocks_x, num_blocks_y):

    rows, cols = images.shape[1:]

    block_size_rows = rows // num_blocks_x
    block_size_cols = cols // num_blocks_y

    images_blocks=[]

    # Slice the image matrix into blocks

    for image in (images):
        sliced_blocks = []
        for i in range(num_blocks_x):
            for j in range(num_blocks_y):
             block = image[i * block_size_rows: (i + 1) * block_size_rows, j * block_size_cols: (j + 1) * block_size_cols]
             sliced_blocks.append(block)

        images_blocks.append(sliced_blocks)
    images_blocks=np.array(images_blocks)

    return images_blocks

def X_bar(img):

 XY=[]
 for i in range (len(img)):
    feature_vector=[]
    for block in range(len(img[i])):
        x_bar=0
        y_bar=0
        denomenator=np.sum(img[i][block])

        numerator1=0
        for x in range(len(img[i][block])):
            for y in range (len(img[i][block][x])):
                numerator1=x*img[i][block][x][y]+numerator1

        numerator2=0
        for y in range(img.shape[3]):
            for x in range (img.shape[2]):
                numerator2=y*img[i][block][x][y]+numerator2

        if denomenator != 0:
           x_bar = numerator1 / denomenator
           y_bar = numerator2 / denomenator
        else:
           x_bar = 0
           y_bar = 0
        feature_vector.append(x_bar)
        feature_vector.append(y_bar)
    feature_vector=np.array(feature_vector)

    XY.append(feature_vector)


 XY=np.array(XY)
 return XY
num_blocks_x =6
num_blocks_y =6

In [51]:
from sklearn.model_selection import train_test_split

#centroid features
Xc = np.concatenate((x1, x2))
y = np.concatenate((y1, y2))
X_block=get_block(Xc,num_blocks_x, num_blocks_y)
X_cent=X_bar(X_block)
X_train, X_test, y_train, y_test = train_test_split(X_cent, y, test_size=0.15, random_state=42)

In [57]:
# Initialize AntColony instance
aco = AntColony(X_cent, n_ants=10, n_iterations=100)

# Run feature selection to select the first 40 features
selected_features = aco.run(n_features=40)

# Print the selected feature indices
print("Selected feature indices:", selected_features)

Selected feature indices: [ 0  1  3  5  7  9 11 13 15 16 17 18 20 21 23 24 25 27 28 29 31 32 34 35
 36 37 38 40 42 44 45 46 47 48 49 51 53 55 56 57]


In [58]:
# Reduce feature space based on selected features
X_selected_train = X_train[:, selected_features]
X_selected_test = X_test[:, selected_features]

In [59]:
# Reshape the input data
X_selected_train_flattened = X_selected_train.reshape(X_selected_train.shape[0], -1)
X_selected_test_flattened = X_selected_test.reshape(X_selected_test.shape[0], -1)


In [60]:
print("Shape of X_train:", X_cent.shape)


Shape of X_train: (531, 72)


In [61]:

import numpy as np

class CustomLDA:
    def __init__(self, n_components=None):
        self.n_components = n_components

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.num_classes = len(self.classes)
        self.num_features = X.shape[1]

        # Compute class means
        self.class_means = np.array([np.mean(X[y == c], axis=0) for c in self.classes])

        # Compute within-class scatter matrix
        within_class_scatter = np.zeros((self.num_features, self.num_features))
        for c in self.classes:
            class_data = X[y == c]
            class_mean_diff = class_data - self.class_means[self.classes.searchsorted(c)]
            within_class_scatter += np.dot(class_mean_diff.T, class_mean_diff)

        # Compute between-class scatter matrix
        overall_mean = np.mean(X, axis=0)
        between_class_scatter = np.zeros((self.num_features, self.num_features))
        for c in self.classes:
            class_mean_diff = self.class_means[self.classes.searchsorted(c)] - overall_mean
            between_class_scatter += len(X[y == c]) * np.outer(class_mean_diff, class_mean_diff)

        # Solve the generalized eigenvalue problem
        eigvals, eigvecs = np.linalg.eig(np.linalg.inv(within_class_scatter).dot(between_class_scatter))
        eig_pairs = [(np.abs(eigvals[i]), eigvecs[:, i]) for i in range(len(eigvals))]
        eig_pairs = sorted(eig_pairs, key=lambda x: x[0], reverse=True)

        # Select top eigenvectors based on number of components
        if self.n_components is not None:
            eig_pairs = eig_pairs[:self.n_components]

        # Compute transformation matrix
        self.w_matrix = np.array([eig_pairs[i][1] for i in range(len(eig_pairs))])

    def transform(self, X):
        return np.dot(X, self.w_matrix.T)

lda = CustomLDA(n_components=2)
lda.fit(X_selected_train_flattened, y_train)
X_reduced_train = lda.transform(X_selected_train_flattened)
X_reduced_test = lda.transform(X_selected_test_flattened)
clf = KNN(k=3)
clf.fit(X_reduced_train, y_train)

# Predictions
predictions = clf.predict(X_reduced_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9875
